In [2]:
#Partical Swarm Optimization
import random
# Objective function: f(x) = x^2
def fitness_function(x):
    return x**2

# Particle class to represent each particle in the swarm
class Particle:
    def __init__(self, min_x, max_x):
        self.position = random.uniform(min_x, max_x)  # Current position
        self.velocity = random.uniform(-1, 1)          # Current velocity
        self.best_position = self.position               # Best position found by the particle
        self.best_fitness = fitness_function(self.position)  # Best fitness value

    def update_velocity(self, global_best_position, inertia_weight, cognitive_coefficient, social_coefficient):
        r1, r2 = random.random(), random.random()
        cognitive_velocity = cognitive_coefficient * r1 * (self.best_position - self.position)
        social_velocity = social_coefficient * r2 * (global_best_position - self.position)
        self.velocity = (inertia_weight * self.velocity) + cognitive_velocity + social_velocity

    def update_position(self, min_x, max_x):
        self.position += self.velocity
        # Ensure the position is within bounds
        self.position = max(min_x, min(self.position, max_x))
        # Update the best position and fitness if needed
        fitness = fitness_function(self.position)
        if fitness < self.best_fitness:  # We want to minimize
            self.best_fitness = fitness
            self.best_position = self.position

# PSO algorithm
def particle_swarm_optimization(pop_size, min_x, max_x, generations, inertia_weight, cognitive_coefficient, social_coefficient):
    # Initialize particles
    swarm = [Particle(min_x, max_x) for _ in range(pop_size)]

    # Global best position initialized to None
    global_best_position = swarm[0].best_position
    global_best_fitness = swarm[0].best_fitness

    for generation in range(generations):
        for particle in swarm:
            # Update global best position
            if particle.best_fitness < global_best_fitness:
                global_best_fitness = particle.best_fitness
                global_best_position = particle.best_position

            # Update particle velocity and position
            particle.update_velocity(global_best_position, inertia_weight, cognitive_coefficient, social_coefficient)
            particle.update_position(min_x, max_x)

        # Print the best fitness in the current generation
        print(f"Generation {generation + 1}: Best solution = {global_best_position}, Fitness = {global_best_fitness}")

    return global_best_position

# Parameters
population_size = 30
min_value = -10
max_value = 10
num_generations = 50
inertia_weight = 0.5
cognitive_coefficient = 1.5
social_coefficient = 1.5

# Run Particle Swarm Optimization
best_solution = particle_swarm_optimization(population_size, min_value, max_value, num_generations, inertia_weight, cognitive_coefficient, social_coefficient)
print(f"Best solution found: {best_solution}, Fitness: {fitness_function(best_solution)}")


Generation 1: Best solution = -0.0022648083368732586, Fitness = 5.1293568027706155e-06
Generation 2: Best solution = -0.0022648083368732586, Fitness = 5.1293568027706155e-06
Generation 3: Best solution = 3.254925925233465e-05, Fitness = 1.0594542778756928e-09
Generation 4: Best solution = 3.254925925233465e-05, Fitness = 1.0594542778756928e-09
Generation 5: Best solution = 3.254925925233465e-05, Fitness = 1.0594542778756928e-09
Generation 6: Best solution = 3.254925925233465e-05, Fitness = 1.0594542778756928e-09
Generation 7: Best solution = 3.254925925233465e-05, Fitness = 1.0594542778756928e-09
Generation 8: Best solution = 3.254925925233465e-05, Fitness = 1.0594542778756928e-09
Generation 9: Best solution = 3.254925925233465e-05, Fitness = 1.0594542778756928e-09
Generation 10: Best solution = 3.254925925233465e-05, Fitness = 1.0594542778756928e-09
Generation 11: Best solution = 3.254925925233465e-05, Fitness = 1.0594542778756928e-09
Generation 12: Best solution = 3.254925925233465e-

In [3]:
#Application: Given a set of stocks determine the optimal portfolio mix to maximize return and minimize risk
import random
import numpy as np

# Fitness function: Calculate Sharpe Ratio (maximize return, minimize risk)
def fitness_function(weights, returns, cov_matrix, risk_free_rate=0):
    # Portfolio return
    portfolio_return = np.dot(weights, returns)

    # Portfolio variance (risk)
    portfolio_variance = np.dot(weights.T, np.dot(cov_matrix, weights))
    portfolio_risk = np.sqrt(portfolio_variance)

    # Sharpe ratio (maximize return, minimize risk)
    sharpe_ratio = (portfolio_return - risk_free_rate) / portfolio_risk
    return sharpe_ratio

# Particle class to represent each particle in the swarm (portfolio weights)
class Particle:
    def __init__(self, num_assets, min_weight, max_weight):
        self.position = np.random.uniform(min_weight, max_weight, num_assets)  # Portfolio weights
        self.velocity = np.random.uniform(-0.1, 0.1, num_assets)  # Portfolio velocity
        self.best_position = self.position  # Best position found by the particle
        self.best_fitness = -float('inf')  # Best fitness value (initially very low)

    def update_velocity(self, global_best_position, inertia_weight, cognitive_coefficient, social_coefficient):
        r1, r2 = random.random(), random.random()
        cognitive_velocity = cognitive_coefficient * r1 * (self.best_position - self.position)
        social_velocity = social_coefficient * r2 * (global_best_position - self.position)
        self.velocity = (inertia_weight * self.velocity) + cognitive_velocity + social_velocity

    def update_position(self, min_weight, max_weight):
        self.position += self.velocity
        # Ensure the weights are between min_weight and max_weight, and the sum of weights is 1
        self.position = np.clip(self.position, min_weight, max_weight)
        self.position /= np.sum(self.position)  # Normalize weights to sum to 1

        # Update the best position and fitness if needed
        fitness = fitness_function(self.position, returns, cov_matrix)
        if fitness > self.best_fitness:  # Maximize fitness
            self.best_fitness = fitness
            self.best_position = self.position

# PSO algorithm to find the optimal portfolio mix
def particle_swarm_optimization(pop_size, num_assets, returns, cov_matrix, min_weight, max_weight, generations, inertia_weight, cognitive_coefficient, social_coefficient):
    # Initialize particles
    swarm = [Particle(num_assets, min_weight, max_weight) for _ in range(pop_size)]

    # Global best position initialized to None
    global_best_position = swarm[0].best_position
    global_best_fitness = swarm[0].best_fitness

    for generation in range(generations):
        for particle in swarm:
            # Update global best position
            if particle.best_fitness > global_best_fitness:
                global_best_fitness = particle.best_fitness
                global_best_position = particle.best_position

            # Update particle velocity and position
            particle.update_velocity(global_best_position, inertia_weight, cognitive_coefficient, social_coefficient)
            particle.update_position(min_weight, max_weight)

        # Print the best fitness in the current generation
        print(f"Generation {generation + 1}: Best solution (weights) = {global_best_position}, Fitness = {global_best_fitness}")

    return global_best_position

# Example usage: optimizing a portfolio of 4 stocks

# Expected returns for the stocks (e.g., annual returns as decimals)
returns = np.array([0.10, 0.12, 0.15, 0.08])

# Covariance matrix of the asset returns (assumed here for 4 stocks)
cov_matrix = np.array([[0.0004, 0.0002, 0.0001, 0.0003],
                       [0.0002, 0.0005, 0.0002, 0.0004],
                       [0.0001, 0.0002, 0.0006, 0.0002],
                       [0.0003, 0.0004, 0.0002, 0.0007]])

# Parameters for PSO
population_size = 30
num_assets = len(returns)
min_weight = 0.05  # Minimum weight per asset
max_weight = 0.40  # Maximum weight per asset
num_generations = 50
inertia_weight = 0.5
cognitive_coefficient = 1.5
social_coefficient = 1.5

# Run Particle Swarm Optimization for portfolio optimization
best_portfolio = particle_swarm_optimization(population_size, num_assets, returns, cov_matrix, min_weight, max_weight, num_generations, inertia_weight, cognitive_coefficient, social_coefficient)

print(f"Best portfolio weights: {best_portfolio}")
print(f"Expected return: {np.dot(best_portfolio, returns)}")
print(f"Portfolio risk (standard deviation): {np.sqrt(np.dot(best_portfolio.T, np.dot(cov_matrix, best_portfolio)))}")


Generation 1: Best solution (weights) = [0.29851723 0.27560854 0.36968902 0.09207892], Fitness = -inf
Generation 2: Best solution (weights) = [ 0.40275154  0.12928887  0.53228079 -0.01018043], Fitness = 7.3171350348624244
Generation 3: Best solution (weights) = [ 0.35201047  0.24886663  0.47705018 -0.03867627], Fitness = 7.335128540085238
Generation 4: Best solution (weights) = [0.29162156 0.24710965 0.45592607 0.00069096], Fitness = 7.337523138188695
Generation 5: Best solution (weights) = [0.29162156 0.24710965 0.45592607 0.00069096], Fitness = 7.337523138188695
Generation 6: Best solution (weights) = [0.29162156 0.24710965 0.45592607 0.00069096], Fitness = 7.337523138188695
Generation 7: Best solution (weights) = [0.30862734 0.22928908 0.43135754 0.03104863], Fitness = 7.337545345444811
Generation 8: Best solution (weights) = [0.30862734 0.22928908 0.43135754 0.03104863], Fitness = 7.337545345444811
Generation 9: Best solution (weights) = [0.33423462 0.20656312 0.42886513 0.01824726